## Concurso de Visualização de Dados da Data Science Academy

Fonte dos dados: sistema SIDRA do IBGE

Repositório deste notebook: https://github.com/jangelojr/dsavisualizacao

In [1]:
import pandas               as pd
import numpy                as np
import plotly.plotly        as py
import plotly.graph_objs    as go
import plotly
import dash
import dash_core_components as dcc
import dash_html_components as html
from plotly import tools

In [2]:
# Tabela de saneamento
tabela1160 = pd.read_excel('Tabela 1160.xlsx')
# Tabela de internações
tabela898  = pd.read_excel('Tabela 898.xlsx')
# Tabela de renda
tabela3734 = pd.read_excel('Tabela 3734.xlsx')
# Tabela de mortalidade infantil
tabela3834 = pd.read_excel('Tabela 3834.xlsx')
# Tabela de frequência escolar líquida
tabela3836 = pd.read_excel('Tabela 3836.xlsx')

In [3]:
# preencher valores vazios com o imediatamente anterior
saneamento   = tabela1160.fillna(method='ffill')
internacoes  =  tabela898.fillna(method='ffill')
renda        = tabela3734.fillna(method='ffill')
mortalidade  = tabela3834.fillna(method='ffill')
freq_escolar = tabela3836.fillna(method='ffill')

In [4]:
# Unir as cinco tabelas
frames = [saneamento, internacoes, renda, mortalidade, freq_escolar]
df = pd.concat(frames)

In [5]:
# Alterar data type
df['Ano'] = df['Ano'].astype('int')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2430 entries, 0 to 134
Data columns (total 4 columns):
Ano                     2430 non-null int64
Unidade da Federação    2430 non-null object
Indicador               2430 non-null object
Valor                   2430 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 94.9+ KB


In [7]:
indicadores = list(df['Indicador'].value_counts().index)

In [8]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets = external_stylesheets)

app.layout = html.Div([
    html.H2(children='Indicadores Sociais - 2011 a 2015'),
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in indicadores],
                value='Até 1/2 salário mínimo'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in indicadores],
                value='Taxa de  frequência escolar líquida'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['Ano'].min(),
        max=df['Ano'].max(),
        value=df['Ano'].max(),
        marks={str(year): str(year) for year in df['Ano'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])

@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 valor_ano):
    dff = df[df['Ano'] == valor_ano]

    return {
        'data': [go.Scatter(
            x=dff[dff['Indicador'] == xaxis_column_name]['Valor'],
            y=dff[dff['Indicador'] == yaxis_column_name]['Valor'],
            text=dff[dff['Indicador'] == yaxis_column_name]['Unidade da Federação'],
            customdata=dff[dff['Indicador'] == yaxis_column_name]['Unidade da Federação'],
            mode='markers',
            marker={
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

def create_time_series(dff, axis_type, title):
    return {
        'data': [go.Scatter(
            x=dff['Ano'],
            y=dff['Valor'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    nome_uf = hoverData['points'][0]['customdata']
    dff = df[df['Unidade da Federação'] == nome_uf]
    dff = dff[dff['Indicador'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(nome_uf, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Unidade da Federação'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicador'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/May/2019 15:32:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2019 15:32:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2019 15:32:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2019 15:32:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2019 15:32:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2019 15:32:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2019 15:32:32] "GET /_favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2019 15:32:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2019 15:32:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2019 15:32:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2019 15:32:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2019 15:32:34] "POST /_dash-update-component HTTP/1.1" 